```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Implicit Equal Weights Particle Filter

This notebook implements prototyping and example/demo of the Implicit Equal Weights Particle Filter (IEWPF).


## Set environment

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation, rc

import pyopencl
import os
import sys

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import CDKLM16, PlotHelper, Common

from SWESimulators import BathymetryAndICs as BC
from SWESimulators import OceanStateNoise


In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
cl_queue = pyopencl.CommandQueue(cl_ctx)
print "Using ", cl_ctx.devices[0].name

# Ensemble

We need an ensemble where each particle
- runs an independent ocean model
- drift a drifter
- applies a localized small-scale error
- observes the drifter position

Needs to be done:
- Initialize models (create netcdf with init, add error with amp 10*q0(?), put drifter into a small area of the 
- make useful plots to evaluate the results
    - Suggestion: 3-line [eta, hu, hv] plot, with truth, ensemble (mean field with individual drifters), mean-square diff?
    - 3x3/4x4/5x5 plot of eta from different ensemble members?
    - Standard animation of a single ensemble member.


## Create initial condition for ensemble:

In [ ]:
# DEFINE PARAMETERS

#Coriolis well balanced reconstruction scheme
nx = 100
ny = 100

dx = 4.0
dy = 4.0

dt = 0.1
g = 9.81
r = 0.0

f = 0.05
beta = 0.0

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
boundaryConditions = Common.BoundaryConditions(2,2,2,2)

# Define which cell index which has lower left corner as position (0,0)
x_zero_ref = 2
y_zero_ref = 2

dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
dataShapeHi = (ny + ghosts[0]+ghosts[2]+1, 
             nx + ghosts[1]+ghosts[3]+1)

eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
eta0_extra = np.zeros(dataShape, dtype=np.float32, order='C')
hv0 = np.zeros(dataShape, dtype=np.float32, order='C');
hu0 = np.zeros(dataShape, dtype=np.float32, order='C');
waterDepth = 10.0
Hi = np.ones(dataShapeHi, dtype=np.float32, order='C')*waterDepth

# Add disturbance:
if True:
    rel_grid_size = nx*1.0/dx
    BC.addBump(eta0, nx, ny, dx, dy, 0.3, 0.5, 0.05*rel_grid_size, validDomain)
    eta0 = eta0*0.3
    BC.addBump(eta0, nx, ny, dx, dy, 0.7, 0.3, 0.10*rel_grid_size, validDomain)
    eta0 = eta0*(-1.3)
    BC.addBump(eta0, nx, ny, dx, dy, 0.15, 0.8, 0.03*rel_grid_size, validDomain)
    eta0 = eta0*1.0
    BC.addBump(eta0, nx, ny, dx, dy, 0.6, 0.75, 0.06*rel_grid_size, validDomain)
    BC.addBump(eta0, nx, ny, dx, dy, 0.2, 0.2, 0.01*rel_grid_size, validDomain)
    eta0 = eta0*(-0.03)
    BC.addBump(eta0_extra, nx, ny, dx, dy, 0.5, 0.5, 0.4*rel_grid_size, validDomain)
    eta0 = eta0 + 0.02*eta0_extra
    BC.initializeBalancedVelocityField(eta0, Hi, hu0, hv0, f, beta, g, nx, ny, dx ,dy, ghosts)
    eta0 = eta0*0.5


#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
#x_coords = np.subtract(x_coords, x_center)
#y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

fig = plt.figure(figsize=(3,3))
plt.imshow(eta0, origin='lower')
plt.colorbar()
plt.contour(eta0, levels=np.linspace(np.min(eta0)*1.1, np.max(eta0)*1.1, 10), colors='black', alpha=0.5)

fig = plt.figure(figsize=(3,3))
plt.imshow(hu0, origin='lower')
plt.colorbar()
fig = plt.figure(figsize=(3,3))
plt.imshow(hv0, origin='lower')
plt.colorbar()


if 'sim' in globals():
    sim.cleanUp()
    
#q0 = 0.00001 # meters amplitude
q0 = 0.03*f/(g*waterDepth)
print "q0: ", q0
print "[f, g, H]", [f, g, waterDepth]
print "f/gH: ", f/(g*waterDepth)
print "gH/f: ", g*waterDepth/f
sim = CDKLM16.CDKLM16(cl_ctx, eta0, hu0, hv0, Hi, \
                      nx, ny, dx, dy, dt, g, f, r, \
                      boundary_conditions=boundaryConditions, \
                      write_netcdf=False, \
                      small_scale_perturbation=True, \
                      small_scale_perturbation_amplitude=q0)
### Animating simulation from the above state

fig = plt.figure(figsize=(8, 10))
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                eta0[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]], 
                                hu0[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]], 
                                hv0[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]])
#plotter.showDrifters(sim.drifters)

T = 100
sub_t = 10*dt
def animate(i):
    if (i>0):
        t = sim.step(sub_t)
    else:
        t = 0.0

    scale = 1.0
    eta1, hu1, hv1 = sim.download()
    plotter.plot(scale*eta1[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]], 
                 scale*hu1[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]], 
                 scale*hv1[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]]);
    #plotter.showDrifters(sim.drifters)
         
    fig.suptitle("CDKLM16 = " + "{:04.0f}".format(t) + " s", fontsize=18)
    
    if (i%10 == 0):
        meanEta = np.mean(eta1[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]])
        minEta = np.min(eta1[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]])
        maxEta = np.max(eta1[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]])
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMean eta: " + str([meanEta, minEta, maxEta]) + \
              "\tMax hu: " + str(np.max(hu1)) + \
              "\tMax hv: " + str(np.max(hv1))
                     
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
eta, hu, hv = sim.download()
fig = plt.figure(figsize=(3,3))
plt.imshow(eta, origin='lower')
plt.colorbar()
plt.contour(eta, levels=np.linspace(np.min(eta)*1.1, np.max(eta)*1.1, 10), colors='black', alpha=0.5)

fig = plt.figure(figsize=(3,3))
plt.imshow(hv, origin='lower')
plt.colorbar()
plt.contour(hv, levels=np.linspace(np.min(hv)*1.1, np.max(hv)*1.1, 10), colors='black', alpha=0.5)
